In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

categoryDF=pd.read_csv("Z:/MSBooks/NLP/FinalProjectFiles/ACDFullData/aspectCategory_new.csv",delimiter='#',encoding = 'utf-8')
categoryDF.head()
train, test = train_test_split(categoryDF, test_size = 0.2)
y_train = train.aspectCategory.unique()
labels = y_train.tolist()

In [3]:
train = train.reset_index(drop='True')
test = test.reset_index(drop='True')

In [4]:
trainData=[]
testData = []
trainLabels = []
testLabels = []
sentiTrainLabels=[]
sentiTestLabels=[]
for id in train.index:
    trainData.append(train.iloc[id,2])
    trainLabels.append(train.iloc[id,3])
    sentiTrainLabels.append(train.iloc[id,1])

for id in test.index:
    testData.append(test.iloc[id,2])
    testLabels.append(test.iloc[id,3])
    sentiTestLabels.append(test.iloc[id,1])

In [5]:
trainLabels = ['missing' if str(x)== 'nan' else x for x in trainLabels]
testLabels = ['missing' if str(x) == 'nan' else x for x in testLabels]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

vectorizer = TfidfVectorizer(min_df=5,
                                 max_df = 0.8,
                                 sublinear_tf=True,
                                 use_idf=True)
train_vectors = vectorizer.fit_transform(trainData)
test_vectors = vectorizer.transform(testData)


classifier_rbf = svm.SVC()
classifier_rbf.fit(train_vectors, trainLabels)
prediction_rbf = classifier_rbf.predict(test_vectors)
senti_classifier_rbf=svm.SVC()
senti_classifier_rbf.fit(train_vectors,sentiTrainLabels)
senti_prediction_rbf = senti_classifier_rbf.predict(test_vectors)

In [7]:
print set(trainLabels)

set([u'hardware&software', u'place&misc', u'software&ease_of_use&hardware', u'price&design&hardware', u'misc', u'design&misc&hardware', u'reachability&scenery', u'hardware', u'design', u'price&misc', u'software&price&hardware', u'hardware&misc', u'design&hardware&price', u'price&design&misc', u'story', u'ease_of_use&hardware', u'performance&story', u'misc&hardware', u'price&ease_of_use', u'design&hardware', u'performance&misc', u'music', u'gui&misc', u'performance', u'scenery', u'price&hardware', u'music&misc', u'price&design', 'missing', u'ease_of_use&design', u'price', u'price&misc&hardware', u'software&design', u'performance&music', u'ease_of_use', u'place&scenery', u'software&misc', u'ease_of_use&price&misc', u'software&hardware', u'software&ease_of_use', u'gui', u'ease_of_use&design&misc', u'software&design&hardware', u'reachability', u'design&misc', u'place', u'ease_of_use&misc', u'scenery&misc', u'ease_of_use&gui&misc', u'software'])


In [8]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
classifier_linear = OneVsRestClassifier(SVC(kernel='linear'))
#classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(train_vectors, trainLabels)
prediction_linear = classifier_linear.predict(test_vectors)

senti_classifier_linear=svm.SVC(kernel='linear')
senti_classifier_linear.fit(train_vectors,sentiTrainLabels)
senti_prediction_linear = senti_classifier_linear.predict(test_vectors)


In [9]:
classifier_liblinear = svm.LinearSVC()
classifier_liblinear.fit(train_vectors, trainLabels)
prediction_liblinear = classifier_liblinear.predict(test_vectors)


senti_classifier_liblinear=svm.LinearSVC()
senti_classifier_liblinear.fit(train_vectors,sentiTrainLabels)
senti_prediction_liblinear = senti_classifier_liblinear.predict(test_vectors)

In [10]:
from sklearn.metrics import f1_score
f1_score(testLabels, prediction_linear, average='micro')  

0.51291512915129156

In [11]:
f1_score(sentiTestLabels, senti_prediction_linear, average='micro')  

0.63929889298892983

In [12]:
print set(prediction_linear)

set([u'price&design&hardware', u'misc', u'hardware', u'design', u'price&misc', u'story', u'ease_of_use&design&misc', u'design&hardware', u'music', u'performance', u'scenery', u'price&hardware', u'missing', u'ease_of_use&design', u'price', u'software&design', u'ease_of_use', u'place&scenery', u'software&hardware', u'software&design&hardware', u'reachability', u'design&misc', u'place', u'software'])


In [38]:
from sklearn import tree
from collections import OrderedDict
mySet = set(trainLabels)
i=0
macros=OrderedDict()
reverseMap=OrderedDict()
for item in mySet:
    macros[item]=i
    reverseMap[i]=item
    i  += 1
trainMacros=[]
testMacros=[]
for item in trainLabels:
    trainMacros.append(macros[item])

    
clf = OneVsRestClassifier(tree.DecisionTreeClassifier())
clf = clf.fit(train_vectors, trainMacros)
result = clf.predict(test_vectors)    

In [39]:
resultList= result.tolist()
resultData=[]
for item in resultList:
    resultData.append(reverseMap[item])
len(resultData)

1084

In [40]:
f1_score(testLabels, resultData, average='micro') 

0.30535055350553508

In [44]:
from sklearn.ensemble import RandomForestClassifier
rfclf = OneVsRestClassifier(RandomForestClassifier())
rfclf.fit(train_vectors,trainMacros)
rfresult = rfclf.predict(test_vectors)
rfresultList = rfresult.tolist()
rfresultData=[]
for item in rfresultList:
    rfresultData.append(reverseMap[item])
f1_score(testLabels,rfresultData,average='micro')

0.43726937269372695

set([u'scenery', u'music', u'price&design', u'software&hardware', 'missing', u'ease_of_use&design', u'price', u'misc', u'software&design&hardware', u'ease_of_use', u'hardware', u'reachability', u'place&scenery', u'design', u'gui&misc', u'performance', u'place', u'hardware&misc', u'software'])
